In [3]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import StratifiedKFold

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 8

# ... (GPU availability and memory growth code from previous response)
def label_images(directory, target_size=(img_width, img_height), max_images_per_class=1000):
    images = []
    labels = []
    class_labels = {'Malignant': 0, 'Benign': 1}

    for class_label, class_index in class_labels.items():
        class_path = os.path.join(directory, class_label)
        image_count = 0
        for root, _, files in os.walk(class_path):
            for filename in files:
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    if image_count < max_images_per_class:
                        file_path = os.path.join(root, filename)
                        image = load_img(file_path, target_size=target_size)  # Load with GPU
                        image = img_to_array(image)
                        image = preprocess_input(image)  # Process the image after loading
                        images.append(image)
                        labels.append(class_index)
                        image_count += 1
                    else:
                        break  # Stop processing this class if max images reached

    return np.array(images), np.array(labels)

# Example usage:
directory_path = 'G:/ChineseCheck/'
X, y = label_images(directory_path, target_size=(img_width, img_height))

# X contains the resized images, and y contains the corresponding labels
print(f"Total Images: {len(X)}")
print(f"Shape of an Image: {X[0].shape}")
print(f"Labels: {y}")

# Split the data into 90% training and 10% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Print the sizes of the training and testing sets
print(f"Training Set: {len(X_train)} samples")
print(f"Testing Set: {len(X_test)} samples")

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

# Assuming you have three classes (malignant, benign, normal)
num_classes = 2
img_width, img_height = 224, 224  # Adjust these dimensions based on your data

def build_resnet_model(learn_rate=0.0001, momentum=0.9):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

    # Freeze all layers except the last three
    for layer in base_model.layers[:-3]:
        layer.trainable = False

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(num_classes, activation='softmax'))

    optimizer = SGD(learning_rate=learn_rate, momentum=momentum)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Build a new ResNet50 model
resnet_model = build_resnet_model()

# Number of folds for k-fold cross-validation
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=110)

for fold, (train_index, val_index) in enumerate(kfold.split(X, y), 1):
    X_train_fold, X_val_fold = X[train_index], X[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    # Convert labels to one-hot encoding
    y_train_fold_one_hot = to_categorical(y_train_fold, num_classes=num_classes)
    y_val_fold_one_hot = to_categorical(y_val_fold, num_classes=num_classes)

    # Train the ResNet50 model
    class_weights = {0: 1.0, 1: 3.0}  # Adjust the weights based on class imbalance

    history = resnet_model.fit(
        X_train_fold,
        y_train_fold_one_hot,
        epochs=10,
        validation_data=(X_val_fold, y_val_fold_one_hot),
        batch_size=8,
        verbose=1,
#         class_weight=class_weights
    )

    # Evaluate the model on the validation set
    val_loss, val_acc = resnet_model.evaluate(X_val_fold, y_val_fold_one_hot, verbose=0)
    print(f"Validation Accuracy for Fold {fold}: {val_acc * 100:.2f}%")

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming you have trained the model 'resnet_model' and loaded the test set 'X_val_fold', 'y_val_fold_one_hot'
evaluate_model_multi_class(resnet_model, X_val_fold, y_val_fold_one_hot)


Total Images: 2000
Shape of an Image: (224, 224, 3)
Labels: [0 0 0 ... 1 1 1]
Training Set: 1800 samples
Testing Set: 200 samples
Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 190s 904ms/step - accuracy: 0.5270 - loss: 1.1030 - val_accuracy: 0.5750 - val_loss: 0.8105
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 179s 898ms/step - accuracy: 0.6898 - loss: 0.5672 - val_accuracy: 0.5925 - val_loss: 0.7366
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 179s 897ms/step - accuracy: 0.7635 - loss: 0.4948 - val_accuracy: 0.6025 - val_loss: 0.7143
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 179s 897ms/step - accuracy: 0.8052 - loss: 0.4268 - val_accuracy: 0.6025 - val_loss: 0.8422
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 178s 892ms/step - accuracy: 0.8525 - loss: 0.3663 - val_accuracy: 0.5825 - val_loss: 0.8076
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 178s 890ms/step - accuracy: 0.8747 - loss: 0.3314 - val_accuracy: 0.5775 - val_loss: 0.8396
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 177s 887ms/step - accuracy: 0.8761 - l

200/200 ━━━━━━━━━━━━━━━━━━━━ 175s 874ms/step - accuracy: 0.9725 - loss: 0.0934 - val_accuracy: 0.9575 - val_loss: 0.1214
Epoch 9/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 176s 880ms/step - accuracy: 0.9825 - loss: 0.0762 - val_accuracy: 0.9500 - val_loss: 0.1601
Epoch 10/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 174s 872ms/step - accuracy: 0.9637 - loss: 0.1043 - val_accuracy: 0.9225 - val_loss: 0.1689
Validation Accuracy for Fold 5: 92.25%


NameError: name 'evaluate_model_multi_class' is not defined

In [4]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Function to calculate and print evaluation metrics for multi-class classification
def evaluate_model_multi_class(model, X, y_true):
    # Predictions
    y_pred = model.predict(X)

    # Convert one-hot encoding to class labels
    y_true_labels = np.argmax(y_true, axis=1)
    y_pred_labels = np.argmax(y_pred, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(y_true_labels, y_pred_labels)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # Confusion Matrix
    cm = confusion_matrix(y_true_labels, y_pred_labels)
    print("Confusion Matrix:")
    print(cm)

    # Classification Report
    print("Classification Report:")
    print(classification_report(y_true_labels, y_pred_labels))

# Assuming you have trained the model 'vgg16_model' and loaded the test set 'X_val_fold', 'y_val_fold_one_hot'
evaluate_model_multi_class(resnet_model, X_val_fold, y_val_fold_one_hot)


13/13 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step
Accuracy: 92.25%
Confusion Matrix:
[[179  21]
 [ 10 190]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.90      0.92       200
           1       0.90      0.95      0.92       200

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

